In [9]:
!pip install --quiet openai google-api-python-client google-auth pandas

In [10]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
import pandas as pd
import openai

# SET THESE VALUES
SPREADSHEET_ID = '16nKBSuG2fzOu453ofrEjAJw32FsVgkM9NoJAVt3UcMo'  # from the Google Sheet URL
AZURE_API_KEY = '8WxLaoodYxa7XSK2rCiWuP3nqwWUShSUVd5FrjEYSqqROfIwc0qzJQQJ99BFAC77bzfXJ3w3AAABACOGweqQ'
AZURE_ENDPOINT = 'https://mindcraft-kapidhwaj-openai-api-key.openai.azure.com/'
AZURE_DEPLOYMENT = 'mindcraft-gpt4o'

# Authenticate with Google Sheets
creds = service_account.Credentials.from_service_account_file(
    'credentials.json',
    scopes=['https://www.googleapis.com/auth/spreadsheets']
)
sheet = build('sheets', 'v4', credentials=creds)

# Authenticate Azure OpenAI
openai.api_type = "azure"
openai.api_base = 'https://mindcraft-kapidhwaj-openai-api-key.openai.azure.com/'
openai.api_version = "2024-12-01-preview"
openai.api_key = '8WxLaoodYxa7XSK2rCiWuP3nqwWUShSUVd5FrjEYSqqROfIwc0qzJQQJ99BFAC77bzfXJ3w3AAABACOGweqQ'


In [11]:
import pandas as pd

def read_biometric_csv(csv_path):
    """
    Reads biometric attendance CSV into DataFrame
    """
    df = pd.read_csv(csv_path)
    df['TimeIn'] = df['TimeIn'].fillna('')
    return df

# Example: Replace with uploaded file path in Colab
df_attendance = read_biometric_csv('biometric_sample.csv')
print(df_attendance)


          Date EmpID      Name TimeIn
0   2025-06-16   E01       Raj  09:13
1   2025-06-16   E02  Abhishek  09:54
2   2025-06-16   E03    Deepak  09:35
3   2025-06-16   E04    Mayank  09:57
4   2025-06-16   E05    Yukthi  09:56
5   2025-06-16   E06     Tanya  10:39
6   2025-06-16   E07     Dhruv  09:58
7   2025-06-16   E08      Riya  09:00
8   2025-06-17   E01       Raj       
9   2025-06-17   E02  Abhishek  09:56
10  2025-06-17   E03    Deepak  09:16
11  2025-06-17   E04    Mayank  09:14
12  2025-06-17   E05    Yukthi  09:07
13  2025-06-17   E06     Tanya  09:08
14  2025-06-17   E07     Dhruv  09:50
15  2025-06-17   E08      Riya  10:40
16  2025-06-18   E01       Raj       
17  2025-06-18   E02  Abhishek       
18  2025-06-18   E03    Deepak  09:34
19  2025-06-18   E04    Mayank  09:37
20  2025-06-18   E05    Yukthi  09:28
21  2025-06-18   E06     Tanya  09:38
22  2025-06-18   E07     Dhruv  09:45
23  2025-06-18   E08      Riya  10:56
24  2025-06-19   E01       Raj       
25  2025-06-

In [12]:
from openai import AzureOpenAI

client = AzureOpenAI(
    api_key="8WxLaoodYxa7XSK2rCiWuP3nqwWUShSUVd5FrjEYSqqROfIwc0qzJQQJ99BFAC77bzfXJ3w3AAABACOGweqQ",
    api_version="2024-12-01-preview",
    azure_endpoint="https://mindcraft-kapidhwaj-openai-api-key.openai.azure.com/"
)

def classify_attendance(name, time_in):
    time_in_str = str(time_in).strip() if time_in else "None"

    messages = [
        {
            "role": "system",
            "content": (
                "You are an AI Attendance Agent. Classify attendance based on time-in.\n"
                "Respond only with:\n"
                "- Present at <time>\n"
                "- Late at <time>\n"
                "- Late Present at <time>\n"
                "- Absent"
            )
        },
        {
            "role": "user",
            "content": f"""
Employee: {name}
Time In: {time_in_str}

Rules:
- Before 09:30 → Present
- Between 09:30 and 10:30 → Late
- After 10:30 → Late Present
- No Time → Absent
"""
        }
    ]

    try:
        response = client.chat.completions.create(
            model="mindcraft-gpt4o",  # use your actual deployment name
            messages=messages
        )
        return response.choices[0].message.content.strip()

    except Exception as e:
        print(f"[ERROR] Classifying {name}: {e}")
        return "Error"


In [13]:
from datetime import datetime

def map_time_to_status(time_in):
    if not time_in or time_in.strip() == "":
        return "🔴"  # Absent
    try:
        time_obj = datetime.strptime(time_in, "%H:%M")
        if time_obj < datetime.strptime("09:30", "%H:%M"):
            return f"✅{time_in}"
        elif time_obj <= datetime.strptime("10:30", "%H:%M"):
            return f"🟡{time_in}"
        else:
            return f"🟠{time_in}"
    except:
        return "❓"  # Unexpected format


In [14]:
def process_calendar_view(csv_path):
    df = read_biometric_csv(csv_path)

    # Get unique list of dates in the file
    dates = sorted(df['Date'].unique())

    # Pivot Data: { (EmpID, Name) : {Date: Status} }
    result_dict = {}
    for _, row in df.iterrows():
        emp_id = row['EmpID']
        name = row['Name']
        date = row['Date']
        time_in = row['TimeIn']
        status = map_time_to_status(time_in)

        key = (emp_id, name)
        if key not in result_dict:
            result_dict[key] = {}
        result_dict[key][date] = status

    # Prepare final DataFrame
    final_data = []
    for (emp_id, name), date_status_map in result_dict.items():
        row = [emp_id, name]
        for d in dates:
            row.append(date_status_map.get(d, "🔴"))  # Mark 🔴 if date missing
        final_data.append(row)

    columns = ["EmpID", "Name"] + dates
    return pd.DataFrame(final_data, columns=columns)


In [15]:
def write_to_google_sheet(df):
    values = [list(df.columns)] + df.values.tolist()
    sheet.spreadsheets().values().update(
        spreadsheetId=SPREADSHEET_ID,
        range="AttendenceSheet!A1",
        valueInputOption="RAW",
        body={"values": values}
    ).execute()


In [16]:
calendar_df = process_calendar_view("biometric_sample.csv")
write_to_google_sheet(calendar_df)

print("✅ Sheet updated in calendar format with colors + time!")


✅ Sheet updated in calendar format with colors + time!


In [17]:
#Test code for LLM
def test_ai_agent(csv_path):
    df = pd.read_csv(csv_path)
    for _, row in df.iterrows():
        name = row['Name']
        time_in = row['TimeIn']
        status = classify_attendance(name, time_in)
        print(f"{name} → {status}")

test_ai_agent("biometric_sample.csv")


Raj → Present at 09:13
Abhishek → Late at 09:54
Deepak → Late at 09:35
Mayank → Late at 09:57
Yukthi → Late at 09:56
Tanya → Late Present at 10:39
Dhruv → Late at 09:58
Riya → Present at 09:00
Raj → Absent
Abhishek → Late at 09:56
Deepak → Present at 09:16
Mayank → Present at 09:14
Yukthi → Present at 09:07
Tanya → Present at 09:08
Dhruv → Late at 09:50
Riya → Late Present at 10:40
Raj → Absent
Abhishek → Absent
Deepak → Late at 09:34
Mayank → Late at 09:37
Yukthi → Present at 09:28
Tanya → Late at 09:38
Dhruv → Late at 09:45
Riya → Late Present at 10:56
Raj → Absent
Abhishek → Late Present at 10:47
Deepak → Present at 09:28
Mayank → Late Present at 10:32
Yukthi → Late at 09:54
Tanya → Present at 09:00
Dhruv → Absent
Riya → Late Present at 10:36
Raj → Late at 09:34
Abhishek → Present at 09:06
Deepak → Late Present at 10:42
Mayank → Late at 09:44
Yukthi → Late Present at 10:45
Tanya → Late Present at 10:40
Dhruv → Late at 09:34
Riya → Present at 09:09


LLM and CSV Read Write is working all fine for now.. Now lets dive deep into **Mailing system**.

In [18]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

EMAIL_ADDRESS = "crazycoder246@gmail.com"
EMAIL_PASSWORD = "qesx gnwh ahht uvkt"  # use app password (not normal password)


In [19]:
employee_emails = {
    "E01": "devamkharbanda440.dk@gmail.com",
    "E02": "poonamkharbanda817@gmail.com",
    "E03": "dev.amkharbanda246@gmail.com",
    "E04": "mayank@example.com"
}
hr_email = "crazycoder246@gmail.com"

In [20]:
def send_hr_summary(attendance_list, hr_email="crazycoder246@gmail.com"):
    present, late, late_present, absent = [], [], [], []

    for record in attendance_list:
        name = record['Name']
        status = record['Status']

        if "✅" in status:
            present.append(f"{name} ({status})")
        elif "🟡" in status:
            late.append(f"{name} ({status})")
        elif "🟠" in status:
            late_present.append(f"{name} ({status})")
        elif "🔴" in status:
            absent.append(f"{name}")

    body = f"""Attendance Summary for Today:

✅ Present:
{chr(10).join(present)}

🟡 Late:
{chr(10).join(late)}

🟠 Late Present:
{chr(10).join(late_present)}

🔴 Absent:
{chr(10).join(absent)}
"""

    msg = MIMEMultipart()
    msg['From'] = EMAIL_ADDRESS
    msg['To'] = hr_email
    msg['Subject'] = "Daily Attendance Summary"
    msg.attach(MIMEText(body, 'plain'))

    with smtplib.SMTP("smtp.gmail.com", 587) as server:
        server.starttls()
        server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
        server.send_message(msg)

    print("✅ HR Summary Email Sent.")


In [21]:
def send_employee_alerts(attendance_list):
    for record in attendance_list:
        emp_id = record['EmpID']
        name = record['Name']
        status = record['Status']

        if "✅" in status:
            continue  # no email for present

        email = employee_emails.get(emp_id)
        if not email:
            print(f"⚠️ No email found for {name}")
            continue

        body = f"""Hi {name},

Your attendance status today was: {status}

This is an automated notification.
Please ensure punctuality.

Regards,
Attendance AI Agent
"""
        msg = MIMEMultipart()
        msg['From'] = EMAIL_ADDRESS
        msg['To'] = email
        msg['Subject'] = "Your Attendance Status"
        msg.attach(MIMEText(body, 'plain'))

        with smtplib.SMTP("smtp.gmail.com", 587) as server:
            server.starttls()
            server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
            server.send_message(msg)

        print(f"📧 Sent alert to {name} ({status})")


In [22]:
attendance_today = [
    {"EmpID": "E01", "Name": "Raj", "Status": "✅09:25"},
    {"EmpID": "E02", "Name": "Abhishek", "Status": "🔴"},
    {"EmpID": "E03", "Name": "Deepak", "Status": "🟠10:45"},
    {"EmpID": "E04", "Name": "Mayank", "Status": "🟡09:32"},
]


In [23]:
send_hr_summary(attendance_today)
send_employee_alerts(attendance_today)


✅ HR Summary Email Sent.
📧 Sent alert to Abhishek (🔴)
📧 Sent alert to Deepak (🟠10:45)
📧 Sent alert to Mayank (🟡09:32)


Mailing Done.. Now lets move on to ChatBot Integration where AI would analyze data and HR could chat with AI on Whatsapp.

In [24]:
!pip install python-telegram-bot==13.15


In [25]:
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters
import openai
import pandas as pd
from datetime import datetime
from googleapiclient.discovery import build
from google.oauth2 import service_account

# --- SETUP ---

# Telegram
TELEGRAM_TOKEN = "7869638478:AAFiW16P7wP9Pv_cStPfNMd5r-kGCv4Yk1g"

# Azure GPT
openai.api_type = "azure"
openai.api_key = "8WxLaoodYxa7XSK2rCiWuP3nqwWUShSUVd5FrjEYSqqROfIwc0qzJQQJ99BFAC77bzfXJ3w3AAABACOGweqQ"
openai.api_base = "https://mindcraft-kapidhwaj-openai-api-key.openai.azure.com/"
openai.api_version = "2024-12-01-preview"
AZURE_DEPLOYMENT = "mindcraft-gpt4o"  # your deployment name

# Google Sheets
SERVICE_ACCOUNT_FILE = 'credentials.json'  # upload this
SPREADSHEET_ID = '16nKBSuG2fzOu453ofrEjAJw32FsVgkM9NoJAVt3UcMo'
SHEET_NAME = 'AttendanceSheet'

SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
creds = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)
sheet = build('sheets', 'v4', credentials=creds)


In [26]:
def fetch_attendance_data():
    result = sheet.spreadsheets().values().get(
        spreadsheetId=SPREADSHEET_ID,
        range=f"AttendenceSheet!A1:Z"
    ).execute()
    values = result.get('values', [])
    return pd.DataFrame(values[1:], columns=values[0]) if values else pd.DataFrame()

from openai import AzureOpenAI

client = AzureOpenAI(
    api_key=openai.api_key,
    api_version=openai.api_version,
    azure_endpoint=openai.api_base
)

def ask_gpt(user_query, df):
    prompt = f"""
You are an AI HR assistant. Use this attendance data to answer questions:
{df.to_string(index=False)}

Question: {user_query}

Answer concisely and clearly based only on the data.
"""
    response = client.chat.completions.create(
        model="mindcraft-gpt4o",  # your deployment name
        messages=[
            {"role": "system", "content": "You're a smart HR assistant for attendance queries."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content.strip()



In [27]:
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters

def handle_message(update, context):
    user_msg = update.message.text
    update.message.reply_text("⏳ Processing your query...")

    try:
        df = fetch_attendance_data()
        reply = ask_gpt(user_msg, df)
        update.message.reply_text(reply)
    except Exception as e:
        update.message.reply_text(f"❌ Error: {str(e)}")

updater = Updater(token=TELEGRAM_TOKEN, use_context=True)
dispatcher = updater.dispatcher
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, handle_message))

print("🤖 Bot is running... Start chatting in Telegram!")
updater.start_polling()
updater.idle()


🤖 Bot is running... Start chatting in Telegram!
